In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

import os
import math

import dask
from dask.distributed import Client
from dask import delayed

import locale 

In [2]:
locale.setlocale(locale.LC_TIME, "es_ES");

# Funciones

In [3]:
def ReportSerieTemporal(sst,TituloFigura):
## Serie temporal anotada con valores maximos y minimos
    
    sst_rolling = sst.rolling(time = 360 , center = True).mean()
    
    tmax = sst.isel(sst.argmax(...))
    tmin = sst.isel(sst.argmin(...))
    
    d_tmax = sst.time.isel(sst.argmax(...))
    d_tmin = sst.time.isel(sst.argmin(...))
    
    #Linear fit
    ind = np.isfinite(sst)
    z = np.polyfit(sst.time.astype(np.int64)[ind],sst[ind], 1)
    Dlinearf = z[0] * sst.time.astype(np.int64) + z[1]
    Dslope=z[0]/1.e-9*24*3600*365*100 #paso a C por siglo
    tTendencia =  "\n %2.2f C "%(sst[-1]-sst[0]) + ' desde el ' + sst.time[0].dt.strftime("%d %B %Y").values + " (Tendencia: " + "%2.2f"%(Dslope) + " C/siglo )"

    tTActual = sst.time[-1].dt.strftime("%d %B %Y").values + " %2.2f $^\circ$C "%(sst[-1].values)
    tTMaxima =  'Temperatura máxima: ' + "%2.2f ºC"%(tmax) + ' el ' + d_tmax.dt.strftime("%d %B %Y").values
    tTMinima =  'Temperatura mínima: ' + "%2.2f ºC"%(tmin) + ' el ' + d_tmin.dt.strftime("%d %B %Y").values
    tPeriodo =  " [" + sstd.time[0].dt.strftime("%d %B %Y").values + " - "+ sstd.time[-1].dt.strftime("%d %B %Y").values + "]"
    
    texto = TituloFigura + tPeriodo + '\n' + tTMaxima + ' - ' + tTMinima + tTendencia
    return texto
    

In [4]:
def FiguraSerieTemporal_anual(sst,TituloFigura):
# Serie temporal por años    

    currentYear = datetime.date.today().year
    sstHist = sst.sel(time=slice("1982-01-01", str(currentYear-1)+"-12-31"))

    df         = sstHist.groupby(sstHist.time.dt.dayofyear).mean().to_dataframe(name="mean")
    df["std"]  = sstHist.groupby(sstHist.time.dt.dayofyear).std().values

    for year, yearda in sst.groupby(sst.time.dt.year):
        df[year] = pd.Series(index=yearda["time"].dt.dayofyear, data=yearda.values)
    
    df.index   = pd.date_range(start='01/Jan/2020', end='31/12/2020', freq='D')  

    currentYear = datetime.date.today().year
    indLastData = np.where(df[currentYear].isnull())[0].tolist()[0]-1

    date_form = DateFormatter("1-%b")
    
    tPeriodo = ' ['+sst.time[0].dt.strftime("%d %B %Y").values + " - "+ sst.time[-1].dt.strftime("%d %B %Y").values + ']'
    tTActual = sst.time[-1].dt.strftime("%d %B %Y").values + " %2.2f (C) "%(sst[-1].values)
    tTMaxima = 'Temperatura máxima: ' + "%2.2f ºC"%(sst.isel(sst.argmax(...)).values)
    tFechaTMaxima = ' el ' + sst.time.isel(sst.argmax(...)).dt.strftime("%d %B %Y").values

    texto=TituloFigura + tPeriodo +'\n' + tTActual + '. ' + tTMaxima + tFechaTMaxima
    return texto

# Inicio

In [5]:
HOME=os.environ['HOME']   
f = open(HOME+'/.env', 'r')
for line in f.readlines():
    Name=line.strip().split('=')[0]
    Content=line.strip().split('=')[-1]
    if Name=='dirData' or Name=='dirAnalisis':
        exec(Name + "=" + "'" + Content + "'")
f.close()

In [6]:
if os.uname().nodename.lower().find('eemmmbp') != -1:
    imagesDir = dirAnalisis + '/SSTGlobalAnalysis/images'
    dataDir   = dirAnalisis + '/SSTGlobalAnalysis/data'    
elif os.uname().nodename.lower().find('sagams') != -1:
    imagesDir = dirAnalisis + '/SSTGlobalAnalysis/images'
    dataDir   = dirAnalisis + '/SSTGlobalAnalysis/data'
elif os.uname().nodename.lower().find('rossby') != -1:
    imagesDir = dirAnalisis + '/SSTGlobalAnalysis/images'
    dataDir   = dirAnalisis + '/SSTGlobalAnalysis/data'

In [7]:
Titulos = ['Oceano Global','Hemisferio norte','Hemisferio sur','AtlanticoNorte', 'Demarcación marina levantino-balear', 'Demarcación marina noratlántica','Demarcación marina canaria','Demarcación sudatlántica','Demarcación Estrecho y Alborán','Iberian Canary Basin']
Titulos_short = ['GO','NH','SH','NAtl','LEB', 'NOR','CAN','SUD','ESA','IBICan']

## Reading data

In [8]:
it=0

In [9]:
titulo = Titulos[it]
titulo_short = Titulos_short[it]

In [10]:
# Daily data
data = xr.open_dataset(dataDir+'/sstd_mean_'+titulo_short+'.nc')
sstd = data.sst
data = xr.open_dataset(dataDir+'/sstd_anom_mean_'+titulo_short+'.nc')
sstd_anom = data.sst

## Times series mean Sea Surface Temperature

In [11]:
Title1  = '- SST promedio en el '+ titulo
texto1=ReportSerieTemporal(sstd,Title1)

Title2  = '- Anomalía SST en el '+ titulo + ' (Anomalía respecto de 1982-1992)'
texto2 = ReportSerieTemporal(sstd_anom,Title2)

Title3  = '- SST en el '+ titulo
texto3 = FiguraSerieTemporal_anual(sstd,Title3)

Title4  = '- Anomalía SST (respecto del periodo 1982-1992), en el '+ titulo
texto4 = FiguraSerieTemporal_anual(sstd_anom,Title4)

print(texto1+'\n'+texto2+'\n'+'\n'+texto3+'\n'+texto4)


- SST promedio en el Oceano Global [01 enero 1982 - 08 octubre 2025]
Temperatura máxima: 19.43 ºC el 24 agosto 2023 - Temperatura mínima: 17.96 ºC el 18 noviembre 1985
 0.59 C  desde el 01 enero 1982 (Tendencia: 1.76 C/siglo )
- Anomalía SST en el Oceano Global (Anomalía respecto de 1982-1992) [01 enero 1982 - 08 octubre 2025]
Temperatura máxima: 0.97 ºC el 23 agosto 2023 - Temperatura mínima: -0.23 ºC el 28 mayo 1985
 0.60 C  desde el 01 enero 1982 (Tendencia: 1.76 C/siglo )

- SST en el Oceano Global [01 enero 1982 - 08 octubre 2025]
08 octubre 2025 18.97 (C) . Temperatura máxima: 19.43 ºC el 24 agosto 2023
- Anomalía SST (respecto del periodo 1982-1992), en el Oceano Global [01 enero 1982 - 08 octubre 2025]
08 octubre 2025 0.68 (C) . Temperatura máxima: 0.97 ºC el 23 agosto 2023
